# Çalışır Hali

In [1]:
import pandas as pd
import numpy as np
import math
from graphviz import Digraph

In [2]:
def calculate_entropy(data, target_col):
    """
    Calculate entropy of the target column
    """
    values = data[target_col].value_counts(normalize=True)
    entropy = -sum(values * np.log2(values))
    return round(entropy, 3)

In [3]:
def calculate_information_gain_ratio(data, feature_col, target_col):
    """
    Calculate information gain ratio of a feature
    """
    total_entropy = calculate_entropy(data, target_col)
    values = data[feature_col].unique()
    weighted_entropy = 0

    for value in values:
        subset = data[data[feature_col] == value]
        weight = len(subset) / len(data)
        subset_entropy = calculate_entropy(subset, target_col)
        weighted_entropy += weight * subset_entropy

    info_gain = total_entropy - weighted_entropy

    # Calculate Split Information
    split_info = -sum(
        (len(data[data[feature_col] == value]) / len(data)) * math.log2(len(data[data[feature_col] == value]) / len(data))
        for value in values
        if len(data[data[feature_col] == value]) > 0
    )

    # Avoid division by zero
    gain_ratio = info_gain / split_info if split_info != 0 else 0
    return round(gain_ratio, 3)

In [4]:
def preprocess_continuous_values(data, feature_col):
    """
    Convert continuous values into categorical (e.g., below threshold, above threshold)
    """
    threshold = data[feature_col].median()
    data[feature_col] = data[feature_col].apply(lambda x: f"below {threshold}" if x <= threshold else f"above {threshold}")

In [5]:
def build_decision_tree(data, target_col, features, depth=0):
    """
    Recursively build a decision tree using C4.5 algorithm
    """
    # Check if the dataset is homogeneous
    if len(data[target_col].unique()) == 1:
        return data[target_col].iloc[0]

    # If no features left, return the most common class
    if not features:
        return data[target_col].mode()[0]

    # Calculate information gain ratio for each feature
    gains = {feature: calculate_information_gain_ratio(data, feature, target_col) for feature in features}
    best_feature = max(gains, key=gains.get)  # Choose the feature with the highest information gain ratio

    # Build the decision tree
    tree = {best_feature: {}}

    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]
        if subset.empty:
            tree[best_feature][value] = data[target_col].mode()[0]
        else:
            subtree = build_decision_tree(
                subset,
                target_col,
                [feat for feat in features if feat != best_feature],
                depth + 1
            )
            tree[best_feature][value] = subtree

    return tree

In [6]:
def print_rules(tree, path=""):
    """
    Convert a decision tree into rules
    """
    if not isinstance(tree, dict):  # Leaf node
        print(f"{path} => {tree}")
        return

    for node, branches in tree.items():
        for value, subtree in branches.items():
            new_path = f"{path} {node} = {value}" if path else f"{node} = {value}"
            print_rules(subtree, new_path)


In [7]:
def visualize_tree(tree, graph=None, parent=None, edge_label=""):
    """
    Visualize a decision tree using Graphviz
    """
    if graph is None:
        graph = Digraph(format='png')
        graph.attr('node', shape='ellipse')

    if isinstance(tree, dict):  # Decision node
        for node, branches in tree.items():
            graph.node(node, label=node)  # Add decision node
            if parent:
                graph.edge(parent, node, label=edge_label)
            for branch_value, subtree in branches.items():
                visualize_tree(subtree, graph, parent=node, edge_label=str(branch_value))
    else:  # Leaf node
        leaf_label = f"{tree}"  # Use leaf value as label
        leaf_node = f"{parent}_{leaf_label}"  # Unique ID for leaf
        graph.node(leaf_node, label=leaf_label, shape='box')
        graph.edge(parent, leaf_node, label=edge_label)

    return graph

In [15]:
# Load dataset
file_path = input("Enter the path to your CSV file: ")
target_col = input("Enter the target column name: ")
data = pd.read_csv(file_path)

Enter the path to your CSV file: /content/C4.5_Dataseti1.csv
Enter the target column name: Decision


In [16]:
for feature in data.columns:
    if feature != target_col and data[feature].dtype in ['float64', 'int64']:
        preprocess_continuous_values(data, feature)

# Define features (exclude target column)
features = [col for col in data.columns if col != target_col]

# Build the decision tree
decision_tree = build_decision_tree(data, target_col, features)

# Visualize the decision tree
dot = visualize_tree(decision_tree)
file_name = input("Enter the file name to save the tree (without extension): ")
dot.render(file_name)  # Save the tree to a file
print(f"Decision tree has been saved as {file_name}.png")
print("Decision Tree Rules:")
print_rules(decision_tree)


Enter the file name to save the tree (without extension): Dataseti1
Decision tree has been saved as Dataseti1.png
Decision Tree Rules:
Outlook = Sunny Humidity = above 80.0 => No
Outlook = Sunny Humidity = below 80.0 => Yes
Outlook = Overcast => Yes
Outlook = Rain Wind = Weak => Yes
Outlook = Rain Wind = Strong => No


# Full Hali

In [ ]:
import pandas as pd
import numpy as np
import math
from graphviz import Digraph

def calculate_entropy(data, target_col):
    """
    Calculate entropy of the target column
    """
    values = data[target_col].value_counts(normalize=True)
    entropy = -sum(values * np.log2(values))
    return round(entropy, 3)

def calculate_information_gain_ratio(data, feature_col, target_col):
    """
    Calculate information gain ratio of a feature
    """
    total_entropy = calculate_entropy(data, target_col)
    values = data[feature_col].unique()
    weighted_entropy = 0

    for value in values:
        subset = data[data[feature_col] == value]
        weight = len(subset) / len(data)
        subset_entropy = calculate_entropy(subset, target_col)
        weighted_entropy += weight * subset_entropy

    info_gain = total_entropy - weighted_entropy

    # Calculate Split Information
    split_info = -sum(
        (len(data[data[feature_col] == value]) / len(data)) * math.log2(len(data[data[feature_col] == value]) / len(data))
        for value in values
        if len(data[data[feature_col] == value]) > 0
    )

    # Avoid division by zero
    gain_ratio = info_gain / split_info if split_info != 0 else 0
    return round(gain_ratio, 3)

def preprocess_continuous_values(data, feature_col):
    """
    Convert continuous values into categorical (e.g., below threshold, above threshold)
    """
    threshold = data[feature_col].median()
    data[feature_col] = data[feature_col].apply(lambda x: f"below {threshold}" if x <= threshold else f"above {threshold}")

def build_decision_tree(data, target_col, features, depth=0):
    """
    Recursively build a decision tree using C4.5 algorithm
    """
    # Check if the dataset is homogeneous
    if len(data[target_col].unique()) == 1:
        return data[target_col].iloc[0]

    # If no features left, return the most common class
    if not features:
        return data[target_col].mode()[0]

    # Calculate information gain ratio for each feature
    gains = {feature: calculate_information_gain_ratio(data, feature, target_col) for feature in features}
    best_feature = max(gains, key=gains.get)  # Choose the feature with the highest information gain ratio

    # Build the decision tree
    tree = {best_feature: {}}

    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]
        if subset.empty:
            tree[best_feature][value] = data[target_col].mode()[0]
        else:
            subtree = build_decision_tree(
                subset,
                target_col,
                [feat for feat in features if feat != best_feature],
                depth + 1
            )
            tree[best_feature][value] = subtree

    return tree

def print_rules(tree, path=""):
    """
    Convert a decision tree into rules
    """
    if not isinstance(tree, dict):  # Leaf node
        print(f"{path} => {tree}")
        return

    for node, branches in tree.items():
        for value, subtree in branches.items():
            new_path = f"{path} {node} = {value}" if path else f"{node} = {value}"
            print_rules(subtree, new_path)

def visualize_tree(tree, graph=None, parent=None, edge_label=""):
    """
    Visualize a decision tree using Graphviz
    """
    if graph is None:
        graph = Digraph(format='png')
        graph.attr('node', shape='ellipse')

    if isinstance(tree, dict):  # Decision node
        for node, branches in tree.items():
            graph.node(node, label=node)  # Add decision node
            if parent:
                graph.edge(parent, node, label=edge_label)
            for branch_value, subtree in branches.items():
                visualize_tree(subtree, graph, parent=node, edge_label=str(branch_value))
    else:  # Leaf node
        leaf_label = f"{tree}"  # Use leaf value as label
        leaf_node = f"{parent}_{leaf_label}"  # Unique ID for leaf
        graph.node(leaf_node, label=leaf_label, shape='box')
        graph.edge(parent, leaf_node, label=edge_label)

    return graph

# Load dataset
file_path = input("Enter the path to your CSV file: ")
target_col = input("Enter the target column name: ")
data = pd.read_csv(file_path)

# Preprocess dataset (convert continuous features to categorical)
for feature in data.columns:
    if feature != target_col and data[feature].dtype in ['float64', 'int64']:
        preprocess_continuous_values(data, feature)

# Define features (exclude target column)
features = [col for col in data.columns if col != target_col]

# Build the decision tree
decision_tree = build_decision_tree(data, target_col, features)

# Visualize the decision tree
dot = visualize_tree(decision_tree)
dot.render("decision_tree_c45")  # Save the tree to a file
print("Decision tree has been saved as decision_tree_c45.png")
print("Decision Tree Rules:")
print_rules(decision_tree)